In [2]:
 from scripts.GetSatellitesGSM import get_artemis_gsm
from scipy.stats import linregress, pearsonr
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import pandas as pd
import numpy as np
import os
# Plots all three events with the 4-hour prerun interval. It makes this interval dashed.

In [4]:
os.makedirs('plots/candidates', exist_ok=True)

keys = ['BX_GSM','BY_GSM','BZ_GSM','VX','VY','VZ','N','T']
names = [r'$B_x$ GSM (nT)', r'$B_y$ GSM (nT)', r'$B_z$ GSM (nT)', r'$V_x$ GSM (km/s)', r'$V_y$ GSM (km/s)', r'$V_z$ GSM (km/s)', r'Proton Density (n/cc)', 'Temperature (K)']

omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('candidate_solar_wind/omni', x)), os.listdir('candidate_solar_wind/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('candidate_solar_wind/artemis', x)), os.listdir('candidate_solar_wind/artemis')))

file_pairs = [(os.path.join('candidate_solar_wind/artemis', artemis_file), os.path.join('candidate_solar_wind/omni', omni_file)) for artemis_file, omni_file in zip(artemisFileList, omniFileList) if not artemis_file.startswith('.') and not 'copy' in artemis_file]

for filename in file_pairs:
    print(f"Processing file: {filename}")
    artemis_data = pd.read_csv(filename[0], delimiter=',', header=0)
    omni_data = pd.read_csv(filename[1], delimiter=',', header=0)

    artemis_data['Time'] = pd.to_datetime(artemis_data['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_data['Time'] = pd.to_datetime(omni_data['Time'], format='%Y-%m-%d %H:%M:%S')

    x_start = artemis_data['Time'].iloc[0]+dt.timedelta(minutes=30)
    x_end = artemis_data['Time'].iloc[-1]

    artemis_truncated = artemis_data[(artemis_data['Time'] >= x_start) & (artemis_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    omni_truncated = omni_data[(omni_data['Time'] >= x_start) & (omni_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    #artemis_truncated = artemis_data.drop(columns=['Unnamed: 0']).reset_index(drop=True)
    #omni_truncated = omni_data.drop(columns=['Unnamed: 0']).reset_index(drop=True)

    fig, ax = plt.subplots(len(keys), 1, figsize=(12, 2*len(keys)), sharex=True)

    for i, (k, n) in enumerate(zip(keys, names)):
        ax[i].plot(artemis_truncated['Time'], artemis_truncated[k], label='Artemis')
        ax[i].plot(omni_truncated['Time'], omni_truncated[k], label='Omni')
        ax[i].set_xlim([x_start, x_end])
        ax[i].set_ylabel(n)
        ax[i].grid(visible=True, which='both', axis='both')

    ax[-1].xaxis.set_major_locator(mdates.HourLocator(interval=3))
    ax[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax[-1].xaxis.set_minor_locator(mdates.HourLocator(interval=1))
    ax[-1].set_xlabel('Time UTC')

    ax[0].set_title(f'Event on {x_start.strftime("%Y-%m-%d")}')
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.savefig(f'plots/candidates_from_filtered/{x_start.strftime("%Y-%m-%d")}.png', dpi=300)
    plt.close()

Processing file: ('candidate_solar_wind/artemis/artemis_2012-01-23_00-00.csv', 'candidate_solar_wind/omni/omni_2012-01-23_00-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2012-01-23_23-00.csv', 'candidate_solar_wind/omni/omni_2012-01-23_23-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2012-11-14_08-00.csv', 'candidate_solar_wind/omni/omni_2012-11-14_08-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2013-06-08_00-00.csv', 'candidate_solar_wind/omni/omni_2013-06-08_00-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2013-06-09_10-00.csv', 'candidate_solar_wind/omni/omni_2013-06-09_10-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2013-07-08_00-00.csv', 'candidate_solar_wind/omni/omni_2013-07-08_00-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2013-12-03_00-00.csv', 'candidate_solar_wind/omni/omni_2013-12-03_00-00.csv')
Processing file: ('candidate_solar_wind/artemis/artemis_2014-0